In [146]:
import pandas as pd
import networkx as nx 
from pprint import pprint
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import re
import numpy as np
import math
from pyvis.network import Network

pd.options.display.max_columns = 50

# Загрузка данных


In [81]:
df_raw = pd.read_csv(r"D:\hh_2006-2020\vacancies_2016.csv")
display(df_raw.head(3))
display(df_raw.shape)

,id,description,key_skills,schedule_id,schedule_name,accept_handicapped,accept_kids,experience_id,experience_name,specializations,...,address_lng,alternate_url,apply_alternate_url,code,department_id,department_name,employment_id,employment_name,prof_classes_found,terms_found
0,16260123,<p>Современная и амбициозная российская компан...,UML\nUse Case Analysis\nнаписание технических ...,fullDay,Полный день,False,False,between1And3,От 1 года до 3 лет,"1.221 Программирование, Разработка 1 Информаци...",...,NaN,https://hh.ru/vacancy/16260123,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,full,Полная занятость,analyst,техническое_образование конкурентная_зарплата ...
1,16260126,"<p><em><strong>Развивающаяся компания, успешно...",Активные продажи\nХолодные продажи\nВедение пе...,fullDay,Полный день,False,False,between3And6,От 3 до 6 лет,"1.225 Продажи 1 Информационные технологии, инт...",...,NaN,https://hh.ru/vacancy/16260126,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,full,Полная занятость,support,кофе team коммуникативные_навыки crm мониторин...
2,16260127,<p><strong>Требования:</strong></p> <p>1.Продв...,NaN,flexible,Гибкий график,True,False,noExperience,Нет опыта,"1.172 Начальный уровень, Мало опыта 1 Информац...",...,NaN,https://hh.ru/vacancy/16260127,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,part,Частичная занятость,manager,гибкий_график желание_работать ms_excel официа...


(332460, 56)

In [103]:
for col in ["id", "employer_id"]:
    print(col)
    df_raw[col] = df_raw[col].astype("Int64")

id
employer_id


# Осмотр

In [106]:
df_raw["code"][df_raw["code"].notna()]

111                                           104-069
112                                           104-069
113                                           104-069
226       Skype: rgkwork Mail: s.nagorny@rgkgroup.com
291                                       Можаев А.С.
                             ...                     
331934                                          D-130
331993                                       IRC36363
332018                                          D-131
332107                                        ПРОДАЖИ
332425                                  инженер связи
Name: code, Length: 4562, dtype: object

In [6]:
df_raw.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332460 entries, 0 to 332459
Data columns (total 56 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         332460 non-null  int64  
 1   description                332460 non-null  object 
 2   key_skills                 81745 non-null   object 
 3   schedule_id                332460 non-null  object 
 4   schedule_name              332460 non-null  object 
 5   accept_handicapped         332460 non-null  bool   
 6   accept_kids                332460 non-null  bool   
 7   experience_id              332460 non-null  object 
 8   experience_name            332460 non-null  object 
 9   specializations            332460 non-null  object 
 10  contacts                   0 non-null       float64
 11  billing_type_id            332460 non-null  object 
 12  billing_type_name          332460 non-null  object 
 13  allow_messages             33

In [15]:
l_column_smoll_unique = []
for col in df_raw.columns:
    unique = df_raw[col].unique()
    if len(unique) < 20:
        print(col, "\t"*(4-((len(col)+1)//8)), len(unique), unique)
    else:
        print(col, "\t"*(4-((len(col)+1)//8)), len(unique))



id 				 332460
description 			 230806
key_skills 			 42173
schedule_id 			 5 ['fullDay' 'flexible' 'remote' 'shift' 'flyInFlyOut']
schedule_name 			 5 ['Полный день' 'Гибкий график' 'Удаленная работа' 'Сменный график'
 'Вахтовый метод']
accept_handicapped 		 2 [False  True]
accept_kids 			 1 [False]
experience_id 			 4 ['between1And3' 'between3And6' 'noExperience' 'moreThan6']
experience_name 		 4 ['От 1 года до 3 лет' 'От 3 до 6 лет' 'Нет опыта' 'Более 6 лет']
specializations 		 60875
contacts 			 1 [nan]
billing_type_id 		 3 ['standard' 'free' 'standard_plus']
billing_type_name 		 3 ['Стандарт' 'Бесплатная' 'Стандарт плюс']
allow_messages 			 2 [ True False]
premium 			 1 [False]
driver_license_types 		 1 [nan]
accept_incomplete_resumes 	 2 [False  True]
employer_id 			 41521
employer_name 			 40912
employer_vacancies_url 		 41521
employer_trusted 		 3 [True False nan]
employer_alternate_url 		 41521
employer_industries 		 6963
response_letter_required 	 2 [False  True]
type_id 			 3

# Постройка графа

Нода вакансия:
id 				 332460
name 				 92470
description 			 230806
key_skills 			 42173 #TODO: возможно в отдельные ноды
schedule_id 			 5 ['fullDay' 'flexible' 'remote' 'shift' 'flyInFlyOut']
schedule_name 			 5 ['Полный день' 'Гибкий график' 'Удаленная работа' 'Сменный график' 'Вахтовый метод']
accept_handicapped 		 2 [False  True]
accept_kids 			 1 [False]
experience_id 			 4 ['between1And3' 'between3And6' 'noExperience' 'moreThan6']
contacts 			 1 [nan]
billing_type_id 		 3 ['standard' 'free' 'standard_plus']
allow_messages 			 2 [ True False]
premium 			 1 [False]
driver_license_types 		 1 [nan]
accept_incomplete_resumes 	 2 [False  True]
response_letter_required 	 2 [False  True]
type_id 			 3 ['open' 'direct' 'anonymous']
salary_from 			 1130
salary_to 			 971
salary_gross 			 1 [nan]
salary_currency 		 11 [nan 'RUR' 'BYR' 'UAH' 'USD' 'KZT' 'EUR' 'AZN' 'UZS' 'KGS' 'GEL']
created_at 			 285317
published_at 			 285317
employment_id 			 5 ['full' 'part' 'project' 'probation' 'volunteer']


Нода Работодателя:
employer_id 			 41521
employer_name 			 40912
employer_trusted 		 3 [True False nan]


Нода специализации:
specializations(list) 		 60875

Нода Индустрия
employer_industries(list) 		 6963

Нода область
area_id 			 2144
area_name 			 2145

Нода адреса
address_city 			 1147
address_street 			 7390
address_building 		 4333
address_description 		 1 [nan]
address_lat 			 20810
address_lng 			 20816

Нода департамента:
department_id 			 144
department_name 		 148

Нода проф классов
prof_classes_found 		 706

Нода найденных определений
terms_found 			 171934


## Подготовка списков

### Список вакансий

In [117]:
df_nodes_vacancies = df_raw.loc[:,["id", "name", "description", "key_skills", 
                                   "schedule_id", "schedule_name", "accept_handicapped", 
                                   "accept_kids", "experience_id", "contacts", "billing_type_id", 
                                   "allow_messages", "premium", "driver_license_types", 
                                   "accept_incomplete_resumes", "response_letter_required", 
                                   "type_id", "salary_from", "salary_to", "salary_gross", 
                                   "salary_currency", "created_at", "published_at", "employment_id"]]
df_nodes_vacancies = df_nodes_vacancies.rename({"name":"label"}, axis="columns")
df_nodes_vacancies = df_nodes_vacancies.set_index("id")

### Список работодателей

In [118]:
df_nodes_employer = df_raw.loc[:,["employer_id", "employer_name", "employer_trusted", "employer_industries"]]\
    .drop_duplicates(["employer_id"]) # TODO: Короче, ничто не уникально! 
df_nodes_employer = df_nodes_employer.set_index("employer_id")

In [147]:
df_edges_employer = df_raw.loc[:,["id", "employer_id"]].rename({"id":"sours", "employer_id":"target"},axis=1)
df_edges_employer

,sours,target
0,16260123,707239
1,16260126,1391634
2,16260127,2139835
3,16260130,2139835
4,16260135,2139835
...,...,...
332455,16029968,1901832
332456,16029976,649011
332457,16029981,1936811
332458,16029984,2206


### Список специализаций

In [133]:
df_nodes_specializations = df_raw.loc[:,["specializations"]]
df_nodes_specializations["specializations"] = df_nodes_specializations["specializations"].str.split("\n")
df_nodes_specializations = df_nodes_specializations.explode("specializations")
df_nodes_specializations = df_nodes_specializations.drop_duplicates().set_index("specializations")


### Список индустрий

In [143]:
df_nodes_employer_industries = df_raw.loc[:,["employer_industries"]].drop_duplicates()
df_nodes_employer_industries["employer_industries"] = df_nodes_employer_industries["employer_industries"].str.split("\n")
df_nodes_employer_industries = df_nodes_employer_industries.explode("employer_industries")
df_nodes_employer_industries = df_nodes_employer_industries.drop_duplicates().set_index("employer_industries")

### Список областей

In [145]:
df_nodes_employer_industries = df_raw.loc[:,["area_id", "area_name"]].drop_duplicates().set_index("area_id")

### Список адресов

### Список департамента

### Список профессиональных областей

### Список найденных определений

# Загрузка данных
